In [35]:
import pandas as pd

from pathlib import Path
import os
import time
from PIL import Image

import torch
from torchvision import transforms

In [ ]:
data_dir = os.path.join(Path.cwd().parent, "data")

IMAGE_SIZE = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.15)),
    transforms.RandomAffine(0, shear=15, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
def load_images(df, transform):
    images = []
    labels = []

    for _, row in df.iterrows():
        img = Image.open(row["image"]).convert("RGB")
        img = transform(img)
        label = row["label_idx"]
        images.append(img)
        labels.append(label)

    return torch.stack(images), torch.tensor(labels)

In [ ]:
df_train = pd.read_csv(data_dir + "\\train_clean_csv.csv")
df_test = pd.read_csv(data_dir + "\\test_clean_csv.csv")
df_val = pd.read_csv(data_dir + "\\val_clean_csv.csv")

In [ ]:
train_imgs, train_labels = load_images(df_train, train_transforms)
test_imgs, test_labels = load_images(df_test, val_test_transforms)
val_imgs, val_labels = load_images(df_val, val_test_transforms)

One image time: 0.11498022079467773


413.92879486083984